# DeutscheWelle Data Engineering Challenge

9.05.2023


# Task 1

## Connect via Gmail API



In [3]:
## Script that dowloads emails from gmail inbox

## from https://developers.google.com/gmail/api/reference/rest/v1/users.messages.attachments/get 
# # Method to download email attachments from gmail api
##
# userId dwdechallenge@gmail.com / me (authenticated user)
# messageId 187ffb2fbf3c1bc5
# id  2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv 
#GET https://gmail.googleapis.com/gmail/v1/users/{userId}/messages/{messageId}/attachments/{id}

In [274]:
#https://stackoverflow.com/questions/25832631/download-attachments-from-gmail-using-gmail-api?answertab=modifieddesc#tab-top 
def get_attachments(service, msg_id):
    try:
        message = service.users().messages().get(userId='me', id=msg_id).execute()

        for part in message['payload']['parts']:
            if part['filename']:
                if 'data' in part['body']:
                    data = part['body']['data']
                else:
                    att_id = part['body']['attachmentId']
                    att = service.users().messages().attachments().get(userId='me', messageId=msg_id,id=att_id).execute()
                    data = att['data']
                file_data = base64.urlsafe_b64decode(data.encode('UTF-8'))
                path = part['filename']
                print(path)
                with open(path, 'wb') as f:
                    f.write(file_data)
    

    except errors.HttpError as error:
        print ('An error occurred: %s') % error

def get_message(service,msg_id):
    try:
        message = service.users().messages().get(userId='me', id=msg_id).execute()
        if message['payload']['mimeType'] == 'multipart/mixed':
            for part in message['payload']['parts']:
                for sub_part in part['parts']:
                    if sub_part['mimeType'] == 'text/plain':
                        data = sub_part['body']['data']
                        break
                if data:
                    break           
        else:
            for part in message['payload']['parts']:
                if part['mimeType'] == 'text/plain':
                    data = part['body']['data']
                    break
        
        content = base64.b64decode(data).decode('utf-8')
        print(content)

        return content    

    except errors.HttpError as error:
        print("An error occured : %s") %error

In [291]:
get_attachments(build('gmail', 'v1', credentials=creds),'187ffb2fbf3c1bc5' )

smime.p7s


In [267]:
## !WORKS! get message IDs
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

creds = Credentials.from_authorized_user_file('../token.json', SCOPES)

try:
    # Call the Gmail API
    
    service = build('gmail', 'v1', credentials=creds)
    results=service.users().messages().list(userId='me').execute()
    #results=service.users().messages().get(userId='me', id=msg_id).execute()
    ids = results.get('messages', [])
    print(type(results))
    print(type(results['messages']))
    print(results['messages'])
    if not ids:
        print('No ids found.')
    
    print('msg_ID:')
    for id in ids:
        print(id['id'])
    
except HttpError as e:
    print('Error response status code : {0}, reason : {1}'.format(e.status_code, e.error_details))
service.close()



<class 'dict'>
<class 'list'>
[{'id': '187ffb33895d80a6', 'threadId': '187ffb33895d80a6'}, {'id': '187ffb2fbf3c1bc5', 'threadId': '187ffb2fbf3c1bc5'}, {'id': '187ffb01677eacf1', 'threadId': '187ffb01677eacf1'}]
msg_ID:
187ffb33895d80a6
187ffb2fbf3c1bc5
187ffb01677eacf1


In [330]:
## get attachment
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from googleapiclient.errors import HttpError
import base64

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

creds = Credentials.from_authorized_user_file('../token.json', SCOPES)
#msg_id='187ffb33895d80a6' # recovery email
msg_id='187ffb2fbf3c1bc5'
try:
    # Call the Gmail API
    
    service = build('gmail', 'v1', credentials=creds)
    #results=service.users().messages().list(userId='me').execute()
    results=service.users().messages().get(userId='me', id=msg_id).execute()
    #ids = results.get('messages', [])
    print((results))
    # print(type(results['messages']))
    #print(results['snippet'])
    print(results['payload'])
    #['partId']) #['filename'])
    print(results['payload']['partId']) #['filename'])
    
    #file_data = base64.urlsafe_b64decode(results.get('data'))
    #.encode('UTF-8'))
    
    # if not ids:
    #     print('No ids found.')
    
    # print('msg_ID:')
    # for id in ids:
    #     print(id['id'])
    
except HttpError as e:
    print('Error response status code : {0}, reason : {1}'.format(e.status_code, e.error_details))
service.close()



{'id': '187ffb2fbf3c1bc5', 'threadId': '187ffb2fbf3c1bc5', 'labelIds': ['UNREAD', 'INBOX'], 'snippet': 'Attachments available until 8. Jun 2023 Click to Download 2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv 5,5 MB Click to Download 2022-09-16-23-04-01-EDT-Historical-', 'payload': {'partId': '', 'mimeType': 'multipart/signed', 'filename': '', 'headers': [{'name': 'Delivered-To', 'value': 'dwdechallenge@gmail.com'}, {'name': 'Received', 'value': 'by 2002:a05:6358:880a:b0:121:447c:4eab with SMTP id hv10csp2797637rwb;        Tue, 9 May 2023 01:48:53 -0700 (PDT)'}, {'name': 'X-Received', 'value': 'by 2002:aa7:d6cf:0:b0:50b:f778:5bf5 with SMTP id x15-20020aa7d6cf000000b0050bf7785bf5mr10172749edr.38.1683622132745;        Tue, 09 May 2023 01:48:52 -0700 (PDT)'}, {'name': 'ARC-Seal', 'value': 'i=1; a=rsa-sha256; t=1683622132; cv=none;        d=google.com; s=arc-20160816;        b=ZqEuX8nH2/ODNCNYz0UFIDsPg5Es4Yo/eqgWoSN3iHU2oEf9aoIRrWcPylhhVM2SFi         Tdfa

In [336]:
real_file_id=GetAttachments(build('gmail', 'v1', credentials=creds), "me", '187ffb2fbf3c1bc5','./' )

./
ANGjdJ98g9oxrLFDP9lBethF6l5TaBmsf7KtEy3Jd6I1nnf7Zoq8Mkkn9KDK4vja9Ev-9LipABz-QMbxJK7fJAwvD4ikjCySDnBqxC7zMTS3d6q13GxI8Kjsn2070iEPOnj9dHtodX-zHAgKBKN3wKcLE1FKil1LM64k6isFQOJ8OcBVKyiSUJo-LH5JMHrM650N2u2Ia5RH6dpIonzB0os_wyuAcdX2HFFTA_kO3-CxSUm86C-iqZhAvGngS4K_0GTFs2QkOU2El90OoC7PFUx0xyxa6NMXgofGM0BHHSbtRm3YawZ3N6h-fPjby2ydg08bvwizgT9N36FLA2lDMcaTb1bJASDG05eNffGhT-7tDRabmuilnnDlxKscUg1QzqWrawQ-KwGiFwfd5TwO
{'partId': '1', 'mimeType': 'application/pkcs7-signature', 'filename': 'smime.p7s', 'headers': [{'name': 'Content-Disposition', 'value': 'attachment; filename=smime.p7s'}, {'name': 'Content-Type', 'value': 'application/pkcs7-signature; name=smime.p7s'}, {'name': 'Content-Transfer-Encoding', 'value': 'base64'}], 'body': {'attachmentId': 'ANGjdJ98g9oxrLFDP9lBethF6l5TaBmsf7KtEy3Jd6I1nnf7Zoq8Mkkn9KDK4vja9Ev-9LipABz-QMbxJK7fJAwvD4ikjCySDnBqxC7zMTS3d6q13GxI8Kjsn2070iEPOnj9dHtodX-zHAgKBKN3wKcLE1FKil1LM64k6isFQOJ8OcBVKyiSUJo-LH5JMHrM650N2u2Ia5RH6dpIonzB0os_wyuAcdX2HFFTA_kO3-CxSUm86C-iqZhAvGngS

In [341]:
GetAttachments(build('gmail', 'v1', credentials=creds), "me", '187ffb2fbf3c1bc5','./' )

./
ANGjdJ_XRusS7h3sNqlUYhn0Yb-bS6A5KqOVcgVjaTfWRJivJ0rXb7214dcEMRYJcuhSmcgJzNEn4tRHTxdufpEzgBYTO3baIbaoSn7iuJ5E9w9ryZIQTREvbIKZB6XgbFqZEKUXJKynzhk9AuNAsIls1cdFrPEbYCrnHm7HgpkN0Ed-37PJPBdPUTYofrvJbse_t26ZnGCkH0XRg6x8lSWDhLHeKOqV8iPx-giDm0c6g6IuF-oYGcqfsvircsl6EeHl177NiGJ93idIT3hODGeAklE2DC_gboBdmxA2ASrOyE33ED2Afm2KSFR0YBBviL9ZmE6n0QKf-rCfQe1eQB01cS8JZ0dXsfDsbr_tMqHnfS56xnnnFtfqn8SIBLNYe2ClffKOJOIAp6Wb8H7L
{'partId': '1', 'mimeType': 'application/pkcs7-signature', 'filename': 'smime.p7s', 'headers': [{'name': 'Content-Disposition', 'value': 'attachment; filename=smime.p7s'}, {'name': 'Content-Type', 'value': 'application/pkcs7-signature; name=smime.p7s'}, {'name': 'Content-Transfer-Encoding', 'value': 'base64'}], 'body': {'attachmentId': 'ANGjdJ_XRusS7h3sNqlUYhn0Yb-bS6A5KqOVcgVjaTfWRJivJ0rXb7214dcEMRYJcuhSmcgJzNEn4tRHTxdufpEzgBYTO3baIbaoSn7iuJ5E9w9ryZIQTREvbIKZB6XgbFqZEKUXJKynzhk9AuNAsIls1cdFrPEbYCrnHm7HgpkN0Ed-37PJPBdPUTYofrvJbse_t26ZnGCkH0XRg6x8lSWDhLHeKOqV8iPx-giDm0c6g6IuF-oYGcqfsvirc

AttributeError: 'bytes' object has no attribute 'uri'

In [319]:
real_file_id
#'ANGjdJ9c7VdvYOnLBIgZdsgI6LT7Bt6_MAi4NcGRtn_TN7fIBMxts8q7icH8qFM58Hv_UPkSZ8UKbaDDMI3zQjX0ZhEi5gr-TkS81AsT_S5TaGSKs5ZZeRp-QqyTBda9-7rr3twwImo9LkrTgFmD5tvt8lB8ZVMx8uj1eX_jl0LIw8nKiwv1nK45-X1_39yJimzAd8h2anenRqLWvwlQ3g9-EBq9XddUe3xLGb4IoOzdjykmg_8gWok5Kt3RxOL0IlcVK5V4MPqR2VgPs_rOxvjFTYlGcL0kwrsAT03dZUQFTkkxfuRiCh49muQkXky53u8MqM1PoXpAbXfiu9eeE_2MoIWftNGJp1eY9i6L5djurvPjKoZCGxO9ahQB9vNOxwxZzXeZvr2p2Yv7Pl_Y'

'ANGjdJ9c7VdvYOnLBIgZdsgI6LT7Bt6_MAi4NcGRtn_TN7fIBMxts8q7icH8qFM58Hv_UPkSZ8UKbaDDMI3zQjX0ZhEi5gr-TkS81AsT_S5TaGSKs5ZZeRp-QqyTBda9-7rr3twwImo9LkrTgFmD5tvt8lB8ZVMx8uj1eX_jl0LIw8nKiwv1nK45-X1_39yJimzAd8h2anenRqLWvwlQ3g9-EBq9XddUe3xLGb4IoOzdjykmg_8gWok5Kt3RxOL0IlcVK5V4MPqR2VgPs_rOxvjFTYlGcL0kwrsAT03dZUQFTkkxfuRiCh49muQkXky53u8MqM1PoXpAbXfiu9eeE_2MoIWftNGJp1eY9i6L5djurvPjKoZCGxO9ahQB9vNOxwxZzXeZvr2p2Yv7Pl_Y'

In [340]:
import base64
from apiclient import errors

def GetAttachments(service, user_id, msg_id, store_dir=""):
    """Get and store attachment from Message with given id.
        Args:
            service: Authorized Gmail API service instance.
            user_id: User's email address. The special value "me"
                can be used to indicate the authenticated user.
            msg_id: ID of Message containing attachment.
            store_dir: The directory used to store attachments.
    """
    try:
        message = service.users().messages().get(userId=user_id, id=msg_id).execute()
        parts = [message['payload']]
        while parts:
            part = parts.pop()
            if part.get('parts'):
                parts.extend(part['parts'])
            if part.get('filename'):
                if 'data' in part['body']:
                    file_data = base64.urlsafe_b64decode(part['body']['data'].encode('UTF-8'))
                    #print(file_data)
                    #self.stdout.write('FileData for %s, %s found! size: %s' % (message['id'], part['filename'], part['size']))
                elif 'attachmentId' in part['body']:
                    attachment = service.users().messages().attachments().get(
                        userId=user_id, messageId=message['id'], id=part['body']['attachmentId']
                    ).execute()
                    file_data = base64.urlsafe_b64decode(attachment['data'].encode('UTF-8'))
                    #print(file_data)
                    #self.stdout.write('FileData for %s, %s found! size: %s' % (message['id'], part['filename'], attachment['size']))
                else:
                    file_data = None
                if file_data:
                    #do some staff, e.g.
                    path = ''.join([store_dir, part['filename']])
                    print(store_dir)
                    file_id=part['body']['attachmentId']
                    print(part['body']['attachmentId'])
                    print(part)
                    file_data = base64.urlsafe_b64decode(attachment['data'].encode('UTF-8'))
                    file = io.BytesIO()
                    downloader = MediaIoBaseDownload(file, file_data)
                    done = False
                    while done is False:
                        status, done = downloader.next_chunk()
                        print(F'Download {int(status.progress() * 100)}.')
                    #print(file_data)
                    # with open(path, 'w') as f:
                    #     f.write(file_data)
                    return file_id
    except errors.HttpError as error:
        print('An error occurred: %s' % error)

In [351]:
##download file
# source https://developers.google.com/drive/api/guides/manage-downloads#python
from __future__ import print_function

import io

import google.auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload


def download_file(real_file_id):
    """Downloads a file
    Args:
        real_file_id: ID of the file to download
    Returns : IO object with location.

    Load pre-authorized user credentials from the environment.
    TODO(developer) - See https://developers.google.com/identity
    for guides on implementing OAuth2 for the application.
    """
    #creds, _ = google.auth.default()
    creds = Credentials.from_authorized_user_file('../token.json', SCOPES)

    try:
        # create drive api client
        #service = build('drive', 'v3', credentials=creds)
        service=build('gmail', 'v1', credentials=creds)
        user_id='me'
        msg_id='187ffb2fbf3c1bc5'
        file_id = real_file_id

        # pylint: disable=maybe-no-member
        # request = service.users().messages().attachments().get(
        #                 userId=user_id, messageId=message['id'], id=part['body']['attachmentId']
        #             ).execute()
        request = service.users().messages().attachments().get(userId=user_id, messageId=msg_id,id=real_file_id).execute()

        #request = service.files().get_media(fileId=file_id)
        file = io.BytesIO()
        downloader = MediaIoBaseDownload(file, request)
        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(F'Download {int(status.progress() * 100)}.')

    except HttpError as error:
        print(F'An error occurred: {error}')
        file = None

    return file.getvalue()


if __name__ == '__main__':
    #download_file(real_file_id='1KuPmvGq8yoYgbfW74OENMCB5H0n_2Jm9')
    download_file(real_file_id=real_file_id)

AttributeError: 'dict' object has no attribute 'uri'

In [362]:
## works? smime7 file. download attachment
from __future__ import print_function
import base64
import os.path
#import oauth2client
from googleapiclient.discovery import build
#from oauth2client import file,tools
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
store_dir=os.getcwd()

def attachment_download():

    #store = oauth2client.file.Storage('credentials_gmail.json')
    #creds = store.get()
    creds = Credentials.from_authorized_user_file('../token.json', SCOPES)

    # if not creds or creds.invalid:
    #     flow = oauth2client.client.flow_from_clientsecrets('client_secrets.json', SCOPES)
    #     creds = oauth2client.tools.run_flow(flow, store)

    try:
        service = build('gmail', 'v1', credentials=creds)
        results = service.users().messages().list(userId='me', labelIds=['INBOX']).execute() # XXXX is label id use INBOX to download from inbox
        messages = results.get('messages', [])
        for message in messages:
            msg = service.users().messages().get(userId='me', id=message['id']).execute()
            for part in msg['payload'].get('parts', ''):

                if part['filename']:
                    if 'data' in part['body']:
                        data = part['body']['data']
                    else:
                        att_id = part['body']['attachmentId']
                        att = service.users().messages().attachments().get(userId='me', messageId=message['id'],id=att_id).execute()
                        data = att['data']
                    file_data = base64.urlsafe_b64decode(data.encode('UTF-8'))

                    filename = part['filename']
                    print(filename)
                    path = os.path.join(store_dir + '/' 'Downloaded_files' + '/' + filename)

                    with open(path, 'wb') as f:
                        f.write(file_data)
                        f.close()
    except Exception as error:
        print(error)

In [365]:
attachment_download()

smime.p7s


In [322]:
real_file_id

'ANGjdJ9c7VdvYOnLBIgZdsgI6LT7Bt6_MAi4NcGRtn_TN7fIBMxts8q7icH8qFM58Hv_UPkSZ8UKbaDDMI3zQjX0ZhEi5gr-TkS81AsT_S5TaGSKs5ZZeRp-QqyTBda9-7rr3twwImo9LkrTgFmD5tvt8lB8ZVMx8uj1eX_jl0LIw8nKiwv1nK45-X1_39yJimzAd8h2anenRqLWvwlQ3g9-EBq9XddUe3xLGb4IoOzdjykmg_8gWok5Kt3RxOL0IlcVK5V4MPqR2VgPs_rOxvjFTYlGcL0kwrsAT03dZUQFTkkxfuRiCh49muQkXky53u8MqM1PoXpAbXfiu9eeE_2MoIWftNGJp1eY9i6L5djurvPjKoZCGxO9ahQB9vNOxwxZzXeZvr2p2Yv7Pl_Y'

In [5]:
import pandas as pd
import psycopg2
import os
import glob



In [6]:
#Write a python script that identifies the correct daily reports

!ls /Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/

daily_repo_filepath= "/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/"

file1='2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv'
files=glob.glob(os.path.join(daily_repo_filepath,"*.csv"))
##date


2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv
2022-09-16-23-04-01-EDT-Historical-Report-ENTRFacebook-2022-06-17--2022-09-17.csv
2022-09-17-23-04-42-EDT-Historical-Report-ENTRFacebook-2022-06-18--2022-09-18.csv
2022-09-18-23-03-32-EDT-Historical-Report-ENTRFacebook-2022-06-19--2022-09-19.csv
2022-09-20-23-03-35-EDT-Historical-Report-ENTRFacebook-2022-06-21--2022-09-21.csv
2022-09-21-23-04-26-EDT-Historical-Report-ENTRFacebook-2022-06-22--2022-09-22.csv
2022-09-23-23-04-15-EDT-Historical-Report-ENTRFacebook-2022-06-24--2022-09-24.csv


In [110]:
files

['/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-18-23-03-32-EDT-Historical-Report-ENTRFacebook-2022-06-19--2022-09-19.csv',
 '/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-16-23-04-01-EDT-Historical-Report-ENTRFacebook-2022-06-17--2022-09-17.csv',
 '/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-21-23-04-26-EDT-Historical-Report-ENTRFacebook-2022-06-22--2022-09-22.csv',
 '/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-17-23-04-42-EDT-Historical-Report-ENTRFacebook-2022-06-18--2022-09-18.csv',
 '/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-20-23-03-35-EDT-Historical-Report-ENTRFacebook-2022-06-21--2022-09-21.csv',
 '/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv',
 '/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-23-23-04-15-EDT-Historical-Report-ENTRFa

In [8]:
data=pd.read_csv(daily_repo_filepath+file1)

In [8]:
data

,Page Name,User Name,Facebook Id,Page Category,Page Admin Top Country,Page Description,Page Created,Likes at Posting,Followers at Posting,Post Created,...,Message,Link,Final Link,Image Text,Link Text,Description,Sponsor Id,Sponsor Name,Sponsor Category,Overperforming Score
0,Onet Styl Życia,onetstylzycia,372341662829366,NEWS_SITE,PL,Najbardziej stylowy serwis w Polsce!,2012-06-05 16:57:21,558850,563725,2022-09-15 21:00:42 EDT,...,"""24. lutego musieliśmy podjąć z mężem ważna i ...",https://www.facebook.com/onetstylzycia/videos/...,NaN,NaN,"""24. lutego wyjechałam z Kijowa. 400 kilometró...",NaN,NaN,NaN,NaN,-0.89
1,Onet Wiadomości,OnetWiadomosci,56024223012,NEWS_SITE,PL,"Jeśli macie jakieś uwagi, chcecie pokazać, jak...",2009-01-13 10:54:16,533815,625545,2022-09-15 19:00:53 EDT,...,"""Pokazują metę, ogromną kasę, przez to, że nie...",https://www.facebook.com/OnetWiadomosci/videos...,NaN,NaN,"""Influencerzy pokazują swój sukces, nie pokazu...",NaN,NaN,NaN,NaN,-4.71
2,FRANCE 24 English,FRANCE24.English,100068681555451,NEWS_SITE,FR,"🌎 International News 24/7\nInfographics, behin...",2009-10-15 12:58:12,1787138,2197295,2022-09-15 17:15:12 EDT,...,Russian forces still occupy vast swathes of Uk...,https://www.facebook.com/FRANCE24.English/vide...,NaN,NaN,War in Ukraine: Citizens of Kramatorsk face di...,NaN,NaN,NaN,NaN,2.50
3,Onet Sport,onetsport,184851814521,SPORTS,PL,Jeden z najpopularniejszych serwisów sportowyc...,2009-11-25 10:54:47,733434,722375,2022-09-15 16:54:08 EDT,...,"W studio pomeczowym podsumujemy to, co wydarzy...",https://www.facebook.com/onetsport/videos/6578...,NaN,NaN,Studio po meczu Lech Poznań - Austria Wiedeń,NaN,1.115661e+14,NaN,NaN,-2.88
4,FRANCE 24 English,FRANCE24.English,100068681555451,NEWS_SITE,FR,"🌎 International News 24/7\nInfographics, behin...",2009-10-15 12:58:12,1787138,2197295,2022-09-15 16:31:26 EDT,...,🇺🇸 The #UnitedStates has taken a step towards ...,https://www.facebook.com/FRANCE24.English/vide...,NaN,NaN,United States Moves to Provide Billions In Mil...,NaN,NaN,NaN,NaN,1.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7069,ZEIT ONLINE,zeitonline,37816894428,NEWS_SITE,DE,ZEIT ONLINE auf\n- Twitter: twitter.com/zeiton...,2008-12-06 11:00:12,825704,882606,2022-06-16 02:08:48 EDT,...,Die Friedensaktivistin Angelika Claußen ist st...,https://www.facebook.com/zeitonline/videos/192...,NaN,NaN,"Waffenlieferungen: ""Es gibt Situationen, da ka...",NaN,NaN,NaN,NaN,-3.97
7070,Noizz Food,NoizzFood,975020859234684,WEBSITE,PL,Zainspiruj się naszymi przepisami i poradami n...,2016-02-09 12:37:25,1092507,1286402,2022-06-16 02:03:49 EDT,...,"Makaron carbonara z groszkiem i łososiem, MNIA...",https://www.facebook.com/NoizzFood/videos/6885...,NaN,NaN,Carbonara z łososiem i groszkiem,NaN,NaN,NaN,NaN,-1.18
7071,FRANCE 24 English,FRANCE24.English,100068681555451,NEWS_SITE,FR,"🌎 International News 24/7\nInfographics, behin...",2009-10-15 12:58:12,1746059,2157715,2022-06-16 01:00:25 EDT,...,"Soon after the Metropol tower block in Abadan,...",https://www.facebook.com/FRANCE24.English/vide...,NaN,NaN,Iranians protest government corruption and neg...,NaN,NaN,NaN,NaN,-2.30
7072,FRANCE 24,FRANCE24,153632746935,TV_CHANNEL,FR,· Twitter : https://f24.my/TWfr\n· YouTube : h...,2009-10-15 12:57:27,10283068,11946422,2022-06-16 01:00:21 EDT,...,🇰🇪 Le Kényan Ferdinand Omanyala est le sprinte...,https://www.facebook.com/FRANCE24/videos/32499...,NaN,NaN,"Sports : A la rencontre de Ferdinand Omanyala,...",NaN,NaN,NaN,NaN,5.09


In [20]:
# connect to postgres database
param_dic = {
    "host"      : "localhost",
    "database"  : "postgres",
    "user"      : "postgres",
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [175]:
conn = connect(param_dic)

def execute_query(conn, query):
    """ Execute a single query """
    connect(param_dic)
    ret = 0 # Return value
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1

    # If this was a select query, return the result
    if 'select' in query.lower():
        ret = cursor.fetchall()
    cursor.close()
    return ret

Connecting to the PostgreSQL database...
Connection successful


In [101]:
#create postgres database

cols=','.join([column.replace(" ", "_").replace("?","").lower() + str(' VARCHAR') for column in data.columns.values])
new_table='new_table'

#query=("DROP TABLE IF EXISTS %s;"%(new_table))
query=("CREATE TABLE IF NOT EXISTS %s(%s)" %(new_table,(cols))) 

execute_query(conn, query)

0

In [109]:
# 2nd option create postgres database
def create_postgres (db,new_table):
    cols=cols=','.join([column.replace(" ", "_").replace("?","").lower() + str(' VARCHAR') for column in data.columns.values])

    conn = connect(param_dic)
    cur = conn.cursor()

    #drop table if already exists
    cur.execute("DROP TABLE IF EXISTS %s;"%(new_table))
    #create database
    cur.execute("CREATE TABLE IF NOT EXISTS %s(%s)" %(new_table,(cols))) 
    conn.commit()
    conn.close()

In [98]:
## data columns to create SQL database 
#VARCHAR for csv input
cols=','.join([column.replace(" ", "_").replace("?","").lower() + str(' VARCHAR') for column in data.columns.values])
cols
# for column in data.columns.values:
#     print(column.replace(" ", "_") + str(type(data[column][0])).replace("class", "").replace("<", "").replace(">", "").replace("'", "").replace("numpy.int64","int").replace("numpy.float64","float").replace("str","text") +',')    

#same with list comprehension
# types=([ column.replace(" ", "_") + str(type(data[column][0])).replace("class", "").replace("<", "").replace(">", "").replace("'", "").replace("numpy.int64","int").replace("numpy.float64","float").replace("str","text") for column in data.columns.values])

'page_name VARCHAR,user_name VARCHAR,facebook_id VARCHAR,page_category VARCHAR,page_admin_top_country VARCHAR,page_description VARCHAR,page_created VARCHAR,likes_at_posting VARCHAR,followers_at_posting VARCHAR,post_created VARCHAR,post_created_date VARCHAR,post_created_time VARCHAR,type VARCHAR,total_interactions VARCHAR,likes VARCHAR,comments VARCHAR,shares VARCHAR,love VARCHAR,wow VARCHAR,haha VARCHAR,sad VARCHAR,angry VARCHAR,care VARCHAR,video_share_status VARCHAR,is_video_owner VARCHAR,post_views VARCHAR,total_views VARCHAR,total_views_for_all_crossposts VARCHAR,video_length VARCHAR,url VARCHAR,message VARCHAR,link VARCHAR,final_link VARCHAR,image_text VARCHAR,link_text VARCHAR,description VARCHAR,sponsor_id VARCHAR,sponsor_name VARCHAR,sponsor_category VARCHAR,overperforming_score_ VARCHAR'

In [13]:
## old create database
#connect database
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres")
cur = conn.cursor()

#create database
cur.execute("""
    CREATE TABLE IF NOT EXISTS reports_varchar(
    page_name VARCHAR,
    user_name VARCHAR,
    facebook_id VARCHAR,
    page_category VARCHAR,
    page_admin_top_country VARCHAR,
    page_description VARCHAR,
    page_created VARCHAR,
    likes_at_posting VARCHAR,
    followers_at_posting VARCHAR,
    post_created VARCHAR,
    post_created_date VARCHAR,
    post_created_time VARCHAR,
    type VARCHAR,
    total_interactions VARCHAR,
    likes VARCHAR,
    comments VARCHAR,
    shares VARCHAR,
    love VARCHAR,
    wow VARCHAR,
    haha VARCHAR,
    sad VARCHAR,
    angry VARCHAR,
    care VARCHAR,
    video_share_status VARCHAR,
    is_video_owner VARCHAR,
    post_views VARCHAR,
    total_views VARCHAR,
    total_views_for_all_crossposts VARCHAR,
    video_length VARCHAR,
    url VARCHAR,
    message VARCHAR,
    link VARCHAR,
    final_link VARCHAR,
    image_VARCHAR VARCHAR,
    link_VARCHAR VARCHAR,
    description VARCHAR,
    sponsor_id VARCHAR,
    sponsor_name VARCHAR,
    sponsor_category VARCHAR,
    overperforming_score_ VARCHAR
)
""")
conn.commit()
conn.close()

In [112]:
## Upload data to the postgres database

def upload_csv_to_postgres(table, csv):
    conn = connect(param_dic)
    cur = conn.cursor()
    try:
        [cur.copy_expert("COPY %s FROM STDIN WITH DELIMITER ',' CSV "%(table), open(csv_file)) for csv_file in files]
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cur.close()
        return 1

    conn.commit()
    conn.close()

In [115]:
table='new_table'

create_postgres(data,table)
upload_csv_to_postgres(table,files)


Connecting to the PostgreSQL database...
Connection successful
Connecting to the PostgreSQL database...
Connection successful


In [122]:
conn = connect(param_dic)
execute_query(conn, 'SELECT Page_Name User_Name FROM new_table')

Connecting to the PostgreSQL database...
Connection successful


[('\ufeffPage Name',),
 ('Onet Styl Życia',),
 ('DW News',),
 ('DW Euromaxx',),
 ('Onet Wiadomości',),
 ('Konbini news',),
 ('Onet Sport',),
 ('G4 Media',),
 ('Onet Sport',),
 ('Noizz Food',),
 ('ENTR ro',),
 ('Noizz Food',),
 ('DW Bundesliga Kick off',),
 ('DW News',),
 ('DW Euromaxx',),
 ('ENTR ro',),
 ('Noizz Food',),
 ('DW Euromaxx (Deutsch)',),
 ('DW Travel',),
 ('DW Culture',),
 ('DW Business',),
 ('DW Travel',),
 ('DW Global Ideas',),
 ('DW Euromaxx',),
 ('DW Stories',),
 ('DW Culture',),
 ('Onet Sport',),
 ('DW Română',),
 ('DW Music',),
 ('Noizz Food',),
 ('DW Euromaxx (Deutsch)',),
 ('Onet Sport',),
 ('DW Travel',),
 ('RFI Romania',),
 ('FRANCE 24 English',),
 ('Onet Sport',),
 ('DW Culture',),
 ('Noizz Food',),
 ('NOIZZ.pl',),
 ('RTP',),
 ('DW Travel',),
 ('DW Reise',),
 ('DW Euromaxx',),
 ('Onet',),
 ('Onet Kultura',),
 ('ENTR fr',),
 ('DW Euromaxx (Deutsch)',),
 ('Noizz Food',),
 ('DW Music',),
 ('DW Culture',),
 ('DW Global Ideas',),
 ('DW Română',),
 ('ENTR en',),
 ('Noi

In [ ]:
##SQL script to process the data and push the data to GIT
## create tables 


# Task 2

#### Have a look at the dwh_dl_facebook_post_insights.csv. You will find there 3 rows for seperate posts. Import the csv to a database table

In [125]:
path_insights='/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/'
db_insights=pd.read_csv(path_insights+'dwh_dl_facebook_post_insights.csv')

In [126]:
db_insights

,insights_created_date,post_activity,post_activity_by_action_type,post_activity_by_action_type_unique,post_activity_unique,post_clicks,post_clicks_by_type,post_clicks_by_type_unique,post_clicks_unique,post_engaged_fan,...,post_video_views_autoplayed,post_video_views_by_distribution_type,post_video_views_clicked_to_play,post_video_views_organic,post_video_views_organic_unique,post_video_views_paid,post_video_views_paid_unique,post_video_views_sound_on,post_video_views_unique,period
0,2019-09-29,67,"{""share"": 6, ""like"": 56, ""comment"": 5}","{""share"": 5, ""like"": 56, ""comment"": 5}",65,384,"{""video play"": 307, ""other clicks"": 71, ""link ...","{""video play"": 303, ""other clicks"": 60, ""link ...",355,31,...,8074,"{""page_owned"": 8264, ""shared"": 48}",238,218,204,8094,7284,1445,7487,lifetime
1,2019-09-29,65,"{""share"": 5, ""like"": 58, ""comment"": 2}","{""share"": 5, ""like"": 58, ""comment"": 2}",61,352,"{""video play"": 265, ""other clicks"": 85}","{""video play"": 265, ""other clicks"": 71}",323,34,...,4467,"{""page_owned"": 4544, ""shared"": 143}",220,269,245,4418,4142,1391,4399,lifetime
2,2019-09-29,93,"{""share"": 9, ""like"": 84}","{""share"": 8, ""like"": 82}",87,327,"{""video play"": 272, ""other clicks"": 53}","{""video play"": 259, ""other clicks"": 48}",303,38,...,5900,"{""shared"": 50, ""page_owned"": 6035}",185,140,116,5945,5445,955,5558,lifetime


In [129]:
files

['/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-18-23-03-32-EDT-Historical-Report-ENTRFacebook-2022-06-19--2022-09-19.csv',
 '/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-16-23-04-01-EDT-Historical-Report-ENTRFacebook-2022-06-17--2022-09-17.csv',
 '/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-21-23-04-26-EDT-Historical-Report-ENTRFacebook-2022-06-22--2022-09-22.csv',
 '/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-17-23-04-42-EDT-Historical-Report-ENTRFacebook-2022-06-18--2022-09-18.csv',
 '/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-20-23-03-35-EDT-Historical-Report-ENTRFacebook-2022-06-21--2022-09-21.csv',
 '/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv',
 '/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/2022-09-23-23-04-15-EDT-Historical-Report-ENTRFa

In [130]:

table='insights_draft'
data=db_insights
files=glob.glob(os.path.join(path_insights,"*.csv"))
#conn = connect(param_dic)
create_postgres(data,table)
upload_csv_to_postgres(table,files)

Connecting to the PostgreSQL database...
Connection successful
Connecting to the PostgreSQL database...
Connection successful


In [151]:
files

['/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/dwh_dl_facebook_post_insights.csv']

### Please write a SQL script to unpack the json in column post_video_view_time_by_age_bucket_and_gender in a new table, while preserving the other columns.

In [ ]:
SELECT 
   id, 
   data::json->'name' as name,
   data::json->'author' ->> 'last_name' as author
FROM books;

In [134]:
conn = connect(param_dic)

sel_column='post_video_view_time_by_age_bucket_and_gender'
table='insights_draft'
execute_query(conn,'SELECT %s FROM %s'%(sel_column,table))

Connecting to the PostgreSQL database...
Connection successful


[('post_video_view_time_by_age_bucket_and_gender',),
 ('{"U.18-24": 92102, "U.25-34": 88501, "U.35-44": 211065, "U.45-54": 287, "U.55-64": 984, "U.65+": 1844, "F.13-17": 1784, "F.18-24": 1937557, "F.25-34": 639440, "F.35-44": 396171, "F.45-54": 246590, "F.55-64": 113451, "F.65+": 87312, "M.13-17": 96475, "M.18-24": 111569485, "M.25-34": 66971129, "M.35-44": 14400899, "M.45-54": 4410015, "M.55-64": 1509493, "M.65+": 2549387}',),
 ('{"U.18-24": 9397, "U.25-34": 11156, "U.35-44": 135338, "U.65+": 5311, "F.13-17": 3193, "F.18-24": 1874800, "F.25-34": 1199817, "F.35-44": 322278, "F.45-54": 304258, "F.55-64": 76856, "F.65+": 36672, "M.13-17": 109554, "M.18-24": 53686462, "M.25-34": 34117456, "M.35-44": 7100589, "M.45-54": 4654955, "M.55-64": 1369288, "M.65+": 1623651}',),
 ('{"U.18-24": 23693, "U.25-34": 260577, "U.35-44": 118480, "U.45-54": 33248, "U.55-64": 6300, "U.65+": 19956, "F.13-17": 23847, "F.18-24": 3580400, "F.25-34": 1657699, "F.35-44": 1190688, "F.45-54": 678030, "F.55-64": 3790

In [176]:
execute_query(conn,'SELECT %s FROM %s'%(sel_column,table))

Connecting to the PostgreSQL database...
Connection successful


[('post_video_view_time_by_age_bucket_and_gender',),
 ('{"U.18-24": 92102, "U.25-34": 88501, "U.35-44": 211065, "U.45-54": 287, "U.55-64": 984, "U.65+": 1844, "F.13-17": 1784, "F.18-24": 1937557, "F.25-34": 639440, "F.35-44": 396171, "F.45-54": 246590, "F.55-64": 113451, "F.65+": 87312, "M.13-17": 96475, "M.18-24": 111569485, "M.25-34": 66971129, "M.35-44": 14400899, "M.45-54": 4410015, "M.55-64": 1509493, "M.65+": 2549387}',),
 ('{"U.18-24": 9397, "U.25-34": 11156, "U.35-44": 135338, "U.65+": 5311, "F.13-17": 3193, "F.18-24": 1874800, "F.25-34": 1199817, "F.35-44": 322278, "F.45-54": 304258, "F.55-64": 76856, "F.65+": 36672, "M.13-17": 109554, "M.18-24": 53686462, "M.25-34": 34117456, "M.35-44": 7100589, "M.45-54": 4654955, "M.55-64": 1369288, "M.65+": 1623651}',),
 ('{"U.18-24": 23693, "U.25-34": 260577, "U.35-44": 118480, "U.45-54": 33248, "U.55-64": 6300, "U.65+": 19956, "F.13-17": 23847, "F.18-24": 3580400, "F.25-34": 1657699, "F.35-44": 1190688, "F.45-54": 678030, "F.55-64": 3790

In [188]:
execute_query(conn, "SELECT %s  FROM %s, data::jsons->'U.18-24' as U.18-24, data::jsons->'U.25-34' as U.25-34"%(sel_column,table))

Connecting to the PostgreSQL database...
Connection successful
Error: syntax error at or near ":"
LINE 1: ..._age_bucket_and_gender  FROM insights_draft,  data:jsons->'U...
                                                             ^



1

In [178]:
execute_query(conn,'SELECT %s FROM %s, json_each(%s)'%(sel_column,table,sel_column))

Connecting to the PostgreSQL database...
Connection successful
Error: function json_each(character varying) does not exist
LINE 1: ...ime_by_age_bucket_and_gender FROM insights_draft, json_each(...
                                                             ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.



1

In [179]:
print(sel_column)
conn = connect(param_dic)
execute_query(conn, "SELECT %s json_data -> '%s' %s, FROM %s;" %(sel_column,sel_column,'foo',table))

post_video_view_time_by_age_bucket_and_gender
Connecting to the PostgreSQL database...
Connection successful
Connecting to the PostgreSQL database...
Connection successful
Error: syntax error at or near "->"
LINE 1: ...ideo_view_time_by_age_bucket_and_gender json_data -> 'post_v...
                                                             ^



1

In [194]:
table

'insights_draft'

In [213]:
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres")
cur = conn.cursor()
cur.execute("""
    SELECT json_data 
        post_video_view_time_by_age_bucket_and_gender -> "U.18-24" as U1
    FROM insights_draft;
    """)
conn.commit()
conn.close()

SyntaxError: syntax error at or near "->"
LINE 3: ...   post_video_view_time_by_age_bucket_and_gender  -> "U.18-2...
                                                             ^


In [193]:
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres")
cur = conn.cursor()
cur.execute("""
    SELECT (post_video_view_time_by_age_bucket_and_gender) from insights_draft;
    """)
conn.commit()
conn.close()

In [183]:
execute_query(conn,'select jsonb_object_keys(%s) from %s;'%(sel_column,table))

Connecting to the PostgreSQL database...
Connection successful
Error: function jsonb_object_keys(character varying) does not exist
LINE 1: select jsonb_object_keys(post_video_view_time_by_age_bucket_...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.



1

In [174]:
table

'insights_draft'

In [172]:
print(sel_column)
conn = connect(param_dic)
execute_query(conn, "SELECT %s -> 'U.18-24' AS U1 FROM %s" %(sel_column,table))

#SELECT post_video_view_time_by_age_bucket_and_gender FROM insights_draft;

#SELECT post_video_view_time_by_age_bucket_and_gender json_data ->> 'U.18-24' AS U1 FROM insights_draft; 

post_video_view_time_by_age_bucket_and_gender
Connecting to the PostgreSQL database...
Connection successful
Error: operator does not exist: character varying -> unknown
LINE 1: ...ECT post_video_view_time_by_age_bucket_and_gender -> 'U.18-2...
                                                             ^
HINT:  No operator matches the given name and argument types. You might need to add explicit type casts.



1

In [ ]:
execute_query(conn,"select 
    json_data #> 
       '{pizzas,1,additionalToppings}'  
        as additional_toppings_2nd_pizza
from test;")

### Write a script for the sums per post for each gender. Additionally create the sum for age 18-34 of all genders.

In [ ]:
# SELECT SUM(Quantity) FROM OrderDetails;
# add where statement WHERE Country='Mexico' WHERE Country='Mexico'
# SELECT * FROM Products WHERE Price BETWEEN 50 AND 60;